In [ ]:
import dash
# import dash_core_components as dcc
from dash import dcc
# import dash_html_components as html
from dash import html
import dash_bootstrap_components as dbc

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from datetime import date

In [ ]:
from jupyter_dash import JupyterDash
from dash.dependencies import Input, Output
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.MATERIA])

IMPORT INCIDENT TICKETS

In [ ]:
# Create initial Dataframe
df_incident = pd.read_csv('/Users/noel.c.joaquin/Desktop/BASE_DATA/INC_YTD_nov2022.csv')
# df_incident.info()

In [ ]:
# Replace NaN values with '01-01-2200' 
df_incident['resolved_at'].fillna('01/01/2200', inplace=True)
df_incident['closed_at'].fillna('01/01/2200', inplace=True)
# df_incident.info()

In [ ]:
# Save new Dataframe 
df_incident.to_csv('/Users/noel.c.joaquin/Desktop/Python/INC_YTD_nov2022.csv', index=False)

In [ ]:
# Recreate Dataframe and conver Date columns to Datetime format
d_parser = lambda x: pd.datetime.strptime(x, '%m/%d/%Y')

df_incident = pd.read_csv('/Users/noel.c.joaquin/Desktop/Python/INC_YTD_nov2022.csv', parse_dates=['opened_at', 'closed_at', 'resolved_at'],
                            date_parser = d_parser , encoding = 'unicode_escape')
                            
# df_incident.head()

IMPORT TASK TICKETS

In [ ]:
# Create initial Dataframe
df_task = pd.read_csv('/Users/noel.c.joaquin/Desktop/BASE_DATA/TASK_YTD_nov2022.csv')
df_task.info()

In [ ]:
# Fill any null values
df_task['closed_at'].fillna('01/01/2200', inplace=True)
# df_task.info()

In [ ]:
# Save new Dataframe 
df_task.to_csv('/Users/noel.c.joaquin/Desktop/Python/TASK_YTD_nov2022.csv', index=False)

In [ ]:
# Create SCTASK DataFrame
d_parser = lambda x: pd.datetime.strptime(x, '%m/%d/%Y')

df_task = pd.read_csv('/Users/noel.c.joaquin/Desktop/Python/TASK_YTD_nov2022.csv', parse_dates=['opened_at', 'closed_at'], date_parser = d_parser, encoding='unicode_escape')

# df_task.head()

GLOBAL VARIABLES

In [ ]:
month_date_today = date.today()
split_month_date_today = str(month_date_today).split('-')
month_first_day = str(month_date_today).replace(str(f"-{split_month_date_today[-1]}"), str(f"-01")) 

CARD COUNTS - INC Opened and Resolved

In [ ]:
def inc_count_or(timeline, state, df):    
    count = 0

    if timeline == 'current-month':
        date_filter_inc_currmonth = (df[state] >= month_first_day)
        count = len(date_filter_inc_currmonth.loc[date_filter_inc_currmonth])
        return count
    elif timeline == 'last-month':
        return count
    elif timeline == 'current-year':
        year_first_day = month_first_day.replace(str(f"-{split_month_date_today[1]}"), str(f"-01"))
        count = df.loc[df[state] >= year_first_day].shape[0]
        return count
    elif timeline == 'ytd':
        count = df.shape[0]
        return count

# INCIDENTS
inc_opened_cm = inc_count_or('current-month', 'opened_at', df_incident)
inc_opened_lm = inc_count_or('last-month', 'opened_at', df_incident)
inc_opened_cy = inc_count_or('current-year', 'opened_at', df_incident)
inc_opened_ytd = inc_count_or('ytd', 'opened_at', df_incident)

inc_resolved_cm = inc_count_or('current-month', 'resolved_at', df_incident)
inc_resolved_lm = inc_count_or('last-month', 'resolved_at', df_incident)
inc_resolved_cy = inc_count_or('current-year', 'resolved_at', df_incident)
inc_resolved_ytd = inc_count_or('ytd', 'resolved_at', df_incident)

# TASKS
# task_opened_lm = inc_count_or('last-month', 'opened_at', df_task)
# task_opened_cm = inc_count_or('current-month', 'opened_at', df_task)
# task_opened_cy = inc_count_or('current-year', 'opened_at', df_task)
# task_opened_ytd = inc_count_or('ytd', 'opened_at', df_task)

# task_resolved_cm = inc_count_or('current-month', 'closed_at', df_task)
# task_resolved_lm = inc_count_or('last-month', 'closed_at', df_task)
# task_resolved_cy = inc_count_or('current-year', 'closed_at', df_task)
# task_resolved_ytd = inc_count_or('ytd', 'closed_at', df_task)

In [ ]:
print(f"inc_opened_cm: {inc_opened_cm}")
print(f"inc_opened_lm: {inc_opened_lm}")
print(f"inc_opened_cy: {inc_opened_cy}")
print(f"inc_opened_ytd: {inc_opened_ytd}")

print("----------------------")

print(f"inc_resolved_cm: {inc_resolved_cm}")
print(f"inc_resolved_lm: {inc_resolved_lm}")
print(f"inc_resolved_cy: {inc_resolved_cy}")
print(f"inc_resolved_ytd: {inc_resolved_ytd}")

Volume of Incidents by Issue Type

In [ ]:
# Get all 'Research Assessment' types
research_assessment_types = df_incident["u_research_assessment"].unique().tolist()
# print(research_assessment_types)

# Get Count for each 'Research Assessment'
research_assessment_count = []
for issue_type in research_assessment_types:
    count = df_incident.loc[df_incident["u_research_assessment"] == issue_type].shape[0]
    research_assessment_count.append(count)

    print(f"issue_type: {issue_type} - {count}")
# print(research_assessment_count)

# Create DataFrame for Research Assessment Count
df_research_assessment = pd.DataFrame({
    "Research Assessment": research_assessment_types,
    "": research_assessment_count
})

df_research_assessment = df_research_assessment.sort_values('', ascending=False)
df_research_assessment

In [ ]:
# Make Bar Chart for df_research_assessment
fig = px.bar(df_research_assessment, x="Research Assessment", y="", barmode="group", text_auto=True)
fig.show()

CARD

In [ ]:
def card_count(title, title_size, data, data_size):
    card = dbc.Card(
            dbc.CardBody(
                [
                    html.H5(title, className="card-title", style={'font-size': title_size}),
                    html.P(data, style={'font-size': data_size, 'font-weight': 'bold'}),
                ]
            )
        )
    
    return card

card_inc_opened = card_count('Incidents Opened', '1.5rem', inc_opened_cm, '5rem')
card_inc_resolved = card_count('Incidents Resolved', '1.5rem', inc_resolved_cm, '5rem')
card_task_opened = card_count('Incidents Opened', '1.5rem', inc_opened_cm, '5rem')
card_task_resolved = card_count('Incidents Resolved', '1.5rem', inc_resolved_cm, '5rem')

cards = dbc.Row(
        [
            dbc.Col(card_inc_opened, width = 3),
            dbc.Col(card_inc_resolved, width = 3),
            dbc.Col(card_task_opened, width = 3),
            dbc.Col(card_task_resolved, width = 3),
        ]
    )


Tickets Opened and Resolved

In [ ]:
def year_month_data(selected_year, state, df):

    initial_year_month_list = df[state].dt.strftime('%Y-%m').unique().tolist()
    year_month_list = []

    for x in initial_year_month_list:
        if selected_year in x:
            year_month_list.append(x)

    # Get available month names for selected year
    get_month_names = df.loc[df[state] > selected_year][state].dt.strftime('%b').unique().tolist()

    return year_month_list, get_month_names
    
inc_ym_2022, inc_mn_2022 = year_month_data('2022', 'opened_at' df_incident)
print(inc_ym_2022)
print(inc_mn_2022)

In [ ]:
def year_month_count_opened(year_month_list, state, df):
    year_month_count_opened = []

    for get_curr_year_month in year_month_list:

        get_curr_month = get_curr_year_month.split('-')[1]
        get_curr_year = get_curr_year_month.split('-')[0]
        get_next_month = int(get_curr_month) + 1
        get_next_year = int(get_curr_year) + 1

        if get_next_month > 12:
            
            get_next_month = 1;
            get_next_year_month = get_curr_year_month.replace(str(f"-{get_curr_month}"), str(f"-{get_next_month}"))
            get_next_year_month = get_next_year_month.replace(str(get_curr_year), str(get_next_year))

            count = df.loc[(df[state] < f"{get_next_year_month}-01") & 
                                    (df[state] >= f"{get_curr_year_month}-01")]

            year_month_count_opened.append(count.shape[0])
        else:
            get_next_year_month = get_curr_year_month.replace(str(f"-{get_curr_month}"), str(f"-{get_next_month}"))

            count = df.loc[(df[state] < f"{get_next_year_month}-01") & 
                                    (df[state] >= f"{get_curr_year_month}-01")]

            year_month_count_opened.append(count.shape[0])

    return year_month_count_opened

inc_ymc_opened_2022 = year_month_count_opened(inc_ym_2022, 'opened_at' ,df_incident)
inc_ymc_resolved_2022 = year_month_count_opened(inc_ym_2022, 'resolved_at' ,df_incident)

task_ymc_opened_2022 = year_month_count_opened(inc_ym_2022, 'opened_at' ,df_task)
task_ymc_resolved_2022 = year_month_count_opened(inc_ym_2022, 'closed_at' ,df_task)

print(f"inc_ymc_opened_2022: {inc_ymc_opened_2022}")
print(f"inc_ymc_resolved_2022: {inc_ymc_resolved_2022}")
print(f"task_ymc_opened_2022: {task_ymc_opened_2022}")
print(f"task_ymc_resolved_2022 : {task_ymc_resolved_2022 }")


In [ ]:
# Create DataFrame for 'opened_at' column based on 'year_month_list' and 'year_month_count'
df_inc_open_and_close = pd.DataFrame({
    "month_year": inc_ym_2022,
    "count_open": inc_ymc_opened_2022,
    "count_resolved": inc_ymc_resolved_2022
})

df_inc_open_and_close

In [ ]:
# Create figure for 'df_inc_open_and_close'
colors = px.colors.qualitative.Set3

fig2 = go.Figure(data=[
    go.Bar(name='Opened', x=inc_mn_2022, y=inc_ymc_opened_2022, text=inc_ymc_opened_2022, marker=dict(color=colors[4])),
    go.Bar(name='Resolved', x=inc_mn_2022, y=inc_ymc_resolved_2022, text=inc_ymc_resolved_2022, marker=dict(color=colors[3])),
])

fig2.show()

TASK TICKETS

In [ ]:
# # Get Opened Task Count for each Year-opened_task = []

# year_month_count_opened_task = []

# for get_curr_year_month in year_month_list:

#     get_curr_month = get_curr_year_month.split('-')[1]
#     get_curr_year = get_curr_year_month.split('-')[0]
#     get_next_month = int(get_curr_month) + 1
#     get_next_year = int(get_curr_year) + 1

#     if get_next_month > 12:
        
#         get_next_month = 1;
#         get_next_year_month = get_curr_year_month.replace(str(f"-{get_curr_month}"), str(f"-{get_next_month}"))
#         get_next_year_month = get_next_year_month.replace(str(get_curr_year), str(get_next_year))

#         count = df_task.loc[(df_task['opened_at'] < f"{get_next_year_month}-01") & 
#                                 (df_task['opened_at'] >= f"{get_curr_year_month}-01")]

#         year_month_count_opened_task.append(count.shape[0])
#     else:
#         get_next_year_month = get_curr_year_month.replace(str(f"-{get_curr_month}"), str(f"-{get_next_month}"))

#         count = df_task.loc[(df_task['opened_at'] < f"{get_next_year_month}-01") & 
#                                 (df_task['opened_at'] >= f"{get_curr_year_month}-01")]

#         year_month_count_opened_task.append(count.shape[0])

# print(year_month_count_opened_task)

In [ ]:
# # Create DataFrame
# df_inc_and_task_open = pd.DataFrame({
#     "month_year": year_month_list,
#     "count_open_inc": year_month_count_opened,
#     "count_open_task": year_month_count_opened_task
# })

# print(df_inc_and_task_open)

In [ ]:
# # Create figure for 'df_inc_and_task_open'
# colors = px.colors.qualitative.Set3

# fig3 = go.Figure(data=[
#     go.Bar(name='Incidents', x=get_month_names, y=year_month_count_opened, text=year_month_count_opened, marker=dict(color=colors[4])),
#     go.Bar(name='Tasks', x=get_month_names, y=year_month_count_opened_task, text=year_month_count_opened_task, marker=dict(color=colors[3])),
# ])

# fig3

In [ ]:
# Filler Pie Chart
df_pie = px.data.tips()
fig_pie = px.pie(df_pie, values='tip', names='day', color='day',
             color_discrete_map={'Thur':'lightcyan',
                                 'Fri':'cyan',
                                 'Sat':'royalblue',
                                 'Sun':'darkblue'})
fig_pie.show()

In [ ]:
# Create server variable with Flask server object for use with gunicorn
server = app.server

# Construct App Layout
app.layout = html.Div(children=[
    html.H1('Analytics Platform Ticket Dashboard'),
    dcc.Tabs(id="tabs-content-input", value='current-month', children=[
        dcc.Tab(label='Current Month', value='current-month'),
        dcc.Tab(label='Last Month', value='last-month'),
        dcc.Tab(label='Current Year', value='current-year'),
        dcc.Tab(label='Year to Date', value='ytd')
    ]),
    html.Div(id='tabs-content-output')
])

@app.callback(Output('tabs-content-output', 'children'),
              Input('tabs-content-input', 'value'))
def render_content(tab):
    if tab == 'current-month':
        return html.Div([
            cards
        ])
    elif tab == 'last-month':
        return html.Div([
            cards
        ])
    elif tab == 'current-year':
        return html.Div([
            cards
        ])
    elif tab == 'ytd':
        return html.Div([
            cards
        ])

app.run_server()